In [2]:
from unsloth import FastLanguageModel
import torch
import os
from dotenv import load_dotenv

load_dotenv()

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

huggingface_token = os.getenv("HUGGING_FACE_TOKEN")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Llama-3.3-70B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=huggingface_token,
)


NameError: name 'os' is not defined